# Limpiar y validar un dataset de ventas

## Dataset con problemas

In [1]:
# Importar librerías
import pandas as pd
import numpy as np

# Crear datos de ejemplo con problemas
ventas = pd.DataFrame({
    'producto': ['A', 'B', None, 'A', 'C'],
    'precio': [100, None, 150, 100, 200],
    'cantidad': [1, 2, None, 1, 3],
    'fecha': ['2024-01-01', None, '2024-01-03', '2024-01-01', 'invalid']
})

print("Datos originales:")
print(ventas)
print(f"Valores faltantes por columna:\n{ventas.isnull().sum()}")

Datos originales:
  producto  precio  cantidad       fecha
0        A   100.0       1.0  2024-01-01
1        B     NaN       2.0        None
2     None   150.0       NaN  2024-01-03
3        A   100.0       1.0  2024-01-01
4        C   200.0       3.0     invalid
Valores faltantes por columna:
producto    1
precio      1
cantidad    1
fecha       1
dtype: int64


## Limpiar datos

In [2]:
def limpiar_datos_ventas(df):
    df_limpio = df.copy()
    
    # 1. Eliminar duplicados
    df_limpio = df_limpio.drop_duplicates()
    
    # 2. Imputar valores faltantes
    df_limpio['precio'] = df_limpio['precio'].fillna(df_limpio['precio'].median())
    df_limpio['cantidad'] = df_limpio['cantidad'].fillna(1)  # Asumir cantidad mínima
    
    # 3. Eliminar filas con producto faltante
    df_limpio = df_limpio.dropna(subset=['producto'])
    
    # 4. Corregir fechas inválidas
    df_limpio['fecha'] = pd.to_datetime(df_limpio['fecha'], errors='coerce')
    df_limpio = df_limpio.dropna(subset=['fecha'])
    
    # 5. Calcular total
    df_limpio['total'] = df_limpio['precio'] * df_limpio['cantidad']
    
    return df_limpio

ventas_limpias = limpiar_datos_ventas(ventas)
print("\nDatos limpios:")
print(ventas_limpias)
print(f"\nRegistros finales: {len(ventas_limpias)}")


Datos limpios:
  producto  precio  cantidad      fecha  total
0        A   100.0       1.0 2024-01-01  100.0

Registros finales: 1


## Validar datos limpios

In [3]:
def validar_ventas_limpias(df):
    validaciones = {
        'sin_faltantes': df.isnull().sum().sum() == 0,
        'precios_positivos': (df['precio'] > 0).all(),
        'cantidades_positivas': (df['cantidad'] > 0).all(),
        'fechas_validas': pd.api.types.is_datetime64_any_dtype(df['fecha']),
        'total_correcto': np.allclose(df['total'], df['precio'] * df['cantidad'])
    }
    
    print("Validaciones:")
    for check, passed in validaciones.items():
        status = "✅" if passed else "❌"
        print(f"  {status} {check}")
    
    return all(validaciones.values())

es_valido = validar_ventas_limpias(ventas_limpias)
print(f"\nDataset válido: {es_valido}")

Validaciones:
  ✅ sin_faltantes
  ✅ precios_positivos
  ✅ cantidades_positivas
  ✅ fechas_validas
  ✅ total_correcto

Dataset válido: True


## Verificación

¿Cuándo deberías eliminar datos faltantes vs imputarlos?
- Depende del tipo de dato y que representa dentro del proceso, del impacto que genera en el negocio y si la imputación puede introducir sesgos en el análisis.

En este ejercicio, se opta por eliminar registros cuando el dato faltante:
    - Es crítico para el análisis y no puede inferirse con seguridad.
    - Rompe la integridad semántica del registro.
    - Impide validaciones clave del dataset.

Ejemplos del ejercicio
    - df_limpio = df_limpio.dropna(subset=['producto'])
    - df_limpio = df_limpio.dropna(subset=['fecha'])

¿Por qué es correcto?
    - Producto = None → No es posible vender “algo desconocido”; imputarlo implicaría crear información ficticia.
    - Fecha inválida o nula → Las fechas son fundamentales para el análisis temporal, agregaciones y reporting, por lo que un valor incorrecto invalida el registro.
    
Entonces, si el dato define qué es el evento o cuándo ocurrió, debe eliminarse.

Imputar datos faltantes

La imputación es apropiada cuando: 	
    - El dato faltante no invalida la existencia del evento
	- Es posible asumir un valor razonable y justificable
    - La imputación no distorsiona los resultados del análisis

Ejemplos del ejercicio
    - df_limpio['precio'] = df_limpio['precio'].fillna(df_limpio['precio'].median())
    - df_limpio['cantidad'] = df_limpio['cantidad'].fillna(1)

¿Por qué es correcto?
    - Precio: utilizar la mediana reduce la influencia de valores extremos (outliers).
    - Cantidad: asumir una unidad corresponde a una regla de negocio conservadora y explícita.

Entonces, si el dato afecta las métricas pero no la validez del evento, puede imputarse aplicando criterio de negocio.


¿Qué tipos de validaciones son más importantes para diferentes tipos de datos?
- Los tipos de validaciones más importantes dependen del rol del dato dentro del proceso analítico. 
Las validaciones de completitud aseguran que el dataset esté listo para modelos, dashboards y cargas a Data Warehouse. 
Las validaciones de dominio evitan errores lógicos, como precios o cantidades inválidas, protegiendo métricas financieras. 
Las validaciones de tipo de dato, especialmente en fechas, garantizan compatibilidad con análisis temporal y reporting. 
Finalmente, las validaciones de consistencia verifican que las métricas calculadas sean correctas, detectando errores de transformación. En conjunto, estas validaciones conforman la base de un pipeline de datos confiable y productivo.

Conclusión:
En el ejercicio se aplica una estrategia de limpieza y validación.
    - Se eliminan datos cuando comprometen la semántica del evento
    - Se imputan valores cuando existen supuestos de negocio razonables
    - Las validaciones cubren completitud, dominio, tipo y consistencia
